## Overview

This notebook is designed to provide a broad overview of Hail's functionality, with emphasis on the functionality to manipulate and query a genetic dataset. We walk through a genome-wide SNP association test, and demonstrate the need to control for confounding caused by population stratification.

Each notebook starts the same: we import the `hail` package and create a [HailContext](https://hail.is/hail/hail.HailContext.html). This object is the entry point for most Hail functionality.

In [ ]:
from hail import *
hc = HailContext()

If the above cell ran without error, we're ready to go! 

Before using Hail, we import some standard Python libraries for use throughout the notebook.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from collections import Counter
from math import log, isnan
from pprint import pprint
%matplotlib inline

 Installing and importing [seaborn](http://seaborn.pydata.org/installing.html) is optional; it just makes the plots prettier.

In [ ]:
# optional
import seaborn

## Check for tutorial data or download if necessary

This cell downloads the necessary data from Google Storage if it isn't found in the current working directory.

In [ ]:
import os
if os.path.isdir('data/1kg.vds') and os.path.isfile('data/1kg_annotations.txt'):
    print('All files are present and accounted for!')
else:
    import sys
    sys.stderr.write('Downloading data (~16MB) from Google Storage...\n')
    import urllib
    import tarfile
    urllib.urlretrieve('https://storage.googleapis.com/hail-tutorial/hail-tutorial-data-v4.tar.gz',
                       'hail-tutorial-data-v4.tar.gz')
    sys.stderr.write('Download finished!\n')
    sys.stderr.write('Extracting...\n')
    tarfile.open('hail-tutorial-data-v4.tar.gz').extractall()
    if not (os.path.isfile('data/1kg.vcf.bgz') and os.path.isfile('data/1kg_annotations.txt')):
        raise RuntimeError('Something went wrong!')
    
    sys.stderr.write('Importing data...\n')
    (hc.import_vcf('data/1kg.vcf.bgz')
     .write('data/1kg.vds'))
    sys.stderr.write('Done!\n')

## Loading data from disk

Hail has its own internal data representation, called a Variant Dataset (VDS). This is both an on-disk file format and a [Python object](https://hail.is/hail/hail.VariantDataset.html). See the [overview](https://hail.is/hail/overview.html) for a complete story. Here, we read a VDS from disk.

This dataset was created by downsampling a public 1000 genomes VCF to about 50 MB.

In [ ]:
vds = hc.read('data/1kg.vds')

## Getting to know our data

It's important to have easy ways to slice, dice, query, and summarize a dataset. Some of these methods are demonstrated below.

The [summarize](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.summarize) method is useful for providing a broad overview of the data contained in a dataset.

In [ ]:
vds.summarize().report()

The [query_variants](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.query_variants) method is the first time we'll see the [Hail expression language](https://hail.is/hail/exprlang.html). The expression language allows for a variety of incredibly expressive queries and computations, but is probably the most complex part of Hail. See the pair of tutorials on the expression language to learn more!

Here, we can use `query_variants` to pull out 5 variants to see what they look like.

In [ ]:
vds.query_variants('variants.take(5)')

There are often several ways to do something in Hail. Here are two ways to peek at the first few sample IDs:

In [ ]:
vds.query_samples('samples.take(5)')

In [ ]:
vds.sample_ids[:5]

There's a similar interface for looking at the genotypes in a dataset. We use [query_genotypes](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.query_genotypes) to look at the first few genotype calls.

In [ ]:
vds.query_genotypes('gs.take(5)')

## Integrate sample annotations

Hail treats variant and sample annotations as first-class citizens. Annotations are usually a critical part of any genetic study. Sample annotations are where you'll store information about sample phenotypes, ancestry, sex, and covariates. Variant annotations can be used to store information like gene membership and functional impact for use in QC or analysis.

In this tutorial, we demonstrate how to take a text file and use it to annotate the samples in a VDS. 

iPython supports various cell "magics". The `%%sh` magic is one which interprets the cell with bash, rather than Python. We can use this to look at the first few lines of our annotation file. This file contains the sample ID, the population and "super-population" designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

In [ ]:
%%sh
head data/1kg_annotations.txt | column -t 

This file can be imported into Hail with [HailContext.import_table](https://hail.is/hail/hail.HailContext.html#hail.HailContext.import_table). This method produces a [KeyTable](https://hail.is/hail/hail.KeyTable.html#hail.KeyTable) object. Think of this as a Pandas or R dataframe that isn't limited by the memory on your machine -- behind the scenes, it's distributed with Spark.

In [ ]:
table = hc.import_table('data/1kg_annotations.txt', impute=True)\
    .key_by('Sample')

A good way to peek at the structure of a `KeyTable` is to look at its `schema`.

In [ ]:
print(table.schema)

The Python `pprint` method makes illegible printouts pretty:

In [ ]:
pprint(table.schema)

Although we used the `%%sh` magic to look at the first lines of the table, there's a better way. We can convert the table to a [Spark DataFrame](https://spark.apache.org/docs/latest/sql-programming-guide.html) and use its `.show()` method:

In [ ]:
table.to_dataframe().show(10)

Now we'll use this table to add sample annotations to our dataset. First, we'll print the schema of the sample annotations already there:

In [ ]:
pprint(vds.sample_schema)

We use the [annotate_samples_table](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.annotate_samples_table) method to join the table with the VDS.

In [ ]:
vds = vds.annotate_samples_table(table, root='sa')

In [ ]:
pprint(vds.sample_schema)

## Query functions and the Hail Expression Language

We start by looking at some statistics of the information in our table. The [query](https://hail.is/hail/hail.KeyTable.html#hail.KeyTable.query) method uses the expression language to aggregate over the rows of the table.

`counter` is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution.

In [ ]:
pprint(table.query('SuperPopulation.counter()'))

`stats` is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [ ]:
pprint(table.query('CaffeineConsumption.stats()'))

However, these metrics aren't perfectly representative of the samples in our dataset. Here's why:

In [ ]:
table.count()

In [ ]:
vds.num_samples

Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can do this with [query_samples](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.query_samples).

In [ ]:
vds.query_samples('samples.map(s => sa.SuperPopulation).counter()')

In [ ]:
pprint(vds.query_samples('samples.map(s => sa.CaffeineConsumption).stats()'))

The functionality demonstrated in the last few cells isn't anything especially new: it's certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like `awk`. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants. 

Here we calculate the counts of each of the 12 possible unique SNPs (4 choices for the reference base * 3 choices for the alternate base). To do this, we need to map the variants to their alternate allele, filter to SNPs, and count by unique ref/alt pair:

In [ ]:
snp_counts = vds.query_variants('variants.map(v => v.altAllele()).filter(aa => aa.isSNP()).counter()')
pprint(Counter(snp_counts).most_common())

It's nice to see that we can actually uncover something biological from this small dataset: we see that these frequencies come in pairs. C/T and G/A are actually the same mutation, just viewed from from opposite strands. Likewise, T/A and A/T are the same mutation on opposite strands. There's a 30x difference between the frequency of C/T and A/T SNPs. Why?

The same Python, R, and Unix tools could do this work as well, but we're starting to hit a wall - the latest [gnomAD release](http://gnomad.broadinstitute.org/) publishes about 250 million variants, and that won't fit in memory on a single computer.

What about genotypes? Hail can query the collection of all genotypes in the dataset, and this is getting large even for our tiny dataset. Our 1,000 samples and 10,000 variants produce 10 million unique genotypes. The gnomAD dataset has about **5 trillion** unique genotypes.

Here we will use the `hist` aggregator to produce and plot a histogram of DP values for genotypes in our thousand genomes dataset.

In [ ]:
dp_hist = vds.query_genotypes('gs.map(g => g.dp).hist(0, 30, 30)')
plt.xlim(0, 31)
plt.bar(dp_hist.binEdges[1:], dp_hist.binFrequencies)
plt.show()

## Quality Control

QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no "push-button" solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the [sample_qc](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.sample_qc) method, which produces a set of useful metrics as sample annotations.

In [ ]:
pprint(vds.sample_schema)

In [ ]:
vds = vds.sample_qc()

In [ ]:
pprint(vds.sample_schema)

Interoperability is a big part of Hail. We can pull out these new metrics to a Pandas dataframe with one line of code:

In [ ]:
df = vds.samples_table().to_pandas()

In [ ]:
df.head()

Plotting the QC metrics is a good place to start.

In [ ]:
plt.clf()
plt.subplot(1, 2, 1)
plt.hist(df["sa.qc.callRate"], bins=np.arange(.75, 1.01, .01))
plt.xlabel("Call Rate")
plt.ylabel("Frequency")
plt.xlim(.75, 1)

plt.subplot(1, 2, 2)
plt.hist(df["sa.qc.gqMean"], bins = np.arange(0, 105, 5))
plt.xlabel("Mean Sample GQ")
plt.ylabel("Frequency")
plt.xlim(0, 105)

plt.tight_layout()
plt.show()

Often, these metrics are correlated.

In [ ]:
plt.scatter(df["sa.qc.dpMean"], df["sa.qc.callRate"], 
            alpha=0.1)
plt.xlabel('Mean DP')
plt.ylabel('Call Rate')
plt.xlim(0, 20)
plt.show()

Removing outliers from the dataset will generally improve association results. We can draw lines on the above plot to indicate outlier cuts. We'll want to remove all samples that fall in the bottom left quadrant.

In [ ]:
plt.scatter(df["sa.qc.dpMean"], df["sa.qc.callRate"], 
            alpha=0.1)
plt.xlabel('Mean DP')
plt.ylabel('Call Rate')
plt.xlim(0, 20)
plt.axhline(0.97, c='k')
plt.axvline(4, c='k')
plt.show()

It's easy to filter when we've got the cutoff values decided:

In [ ]:
vds = vds.filter_samples_expr('sa.qc.dpMean >= 4 && sa.qc.callRate >= 0.97')
print('After filter, %d/1000 samples remain.' % vds.num_samples)

Next is genotype QC. To start, we'll print the post-sample-QC call rate. It's actually gone _up_ since the initial summary - dropping low-quality samples disproportionally removed missing genotypes.

In [ ]:
call_rate = vds.query_genotypes('gs.fraction(g => g.isCalled)')
print('pre QC call rate is %.3f' % call_rate)

It's a good idea to filter out genotypes where the reads aren't where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In [ ]:
filter_condition_ab = '''let ab = g.ad[1] / g.ad.sum() in
                         ((g.isHomRef && ab <= 0.1) ||
                          (g.isHet && ab >= 0.25 && ab <= 0.75) ||
                          (g.isHomVar && ab >= 0.9))'''
vds = vds.filter_genotypes(filter_condition_ab)

In [ ]:
post_qc_call_rate = vds.query_genotypes('gs.fraction(g => g.isCalled)')
print('post QC call rate is %.3f' % post_qc_call_rate)

Variant QC is a bit more of the same: we can use the [variant_qc](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.variant_qc) method to produce a variety of useful statistics, plot them, and filter.

In [ ]:
pprint(vds.variant_schema)

The [cache](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.cache) is used to optimize some of the downstream operations.

In [ ]:
vds = vds.variant_qc().cache()

In [ ]:
pprint(vds.variant_schema)

In [ ]:
variant_df = vds.variants_table().to_pandas()

plt.clf()
plt.subplot(2, 2, 1)
variantgq_means = variant_df["va.qc.gqMean"]
plt.hist(variantgq_means, bins = np.arange(0, 84, 2))
plt.xlabel("Variant Mean GQ")
plt.ylabel("Frequency")
plt.xlim(0, 80)

plt.subplot(2, 2, 2)
variant_mleaf = variant_df["va.qc.AF"]
plt.hist(variant_mleaf, bins = np.arange(0, 1.05, .025))
plt.xlabel("Minor Allele Frequency")
plt.ylabel("Frequency")
plt.xlim(0, 1)

plt.subplot(2, 2, 3)
plt.hist(variant_df['va.qc.callRate'], bins = np.arange(0, 1.05, .01))
plt.xlabel("Variant Call Rate")
plt.ylabel("Frequency")
plt.xlim(.5, 1)

plt.subplot(2, 2, 4)
plt.hist(variant_df['va.qc.pHWE'], bins = np.arange(0, 1.05, .025))
plt.xlabel("Hardy-Weinberg Equilibrium p-value")
plt.ylabel("Frequency")
plt.xlim(0, 1)

plt.tight_layout()
plt.show()


These statistics actually look pretty good: we don't need to filter this dataset. Most datasets require thoughtful quality control, though. The [filter_variants_expr](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.filter_variants_expr) method can help!

## Let's do a GWAS!

First, we need to restrict to variants that are : 

 - common (we'll use a cutoff of 1%)
 - uncorrelated (not in linkage disequilibrium)
 
Both of these are easy in Hail.

In [ ]:
common_vds = (vds
              .filter_variants_expr('va.qc.AF > 0.01')
              .ld_prune(memory_per_core=256, num_cores=4))

In [ ]:
common_vds.count()

These filters removed about 15% of sites (we started with a bit over 10,000). This is _NOT_ representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we'd expect by chance.

In Hail, the association tests accept sample annotations for the sample phenotype and covariates. Since we've already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [ ]:
gwas = common_vds.linreg(['sa.CaffeineConsumption'])
pprint(gwas.variant_schema)    

Looking at the bottom of the above printout, you can see the linear regression adds new variant annotations for the beta, standard error, t-statistic, and p-value.

In [ ]:
def qqplot(pvals, xMax, yMax):
    spvals = sorted(filter(lambda x: x and not(isnan(x)), pvals))
    exp = [-log(float(i) / len(spvals), 10) for i in np.arange(1, len(spvals) + 1, 1)]
    obs = [-log(p, 10) for p in spvals]
    plt.clf()
    plt.scatter(exp, obs)
    plt.plot(np.arange(0, max(xMax, yMax)), c="red")
    plt.xlabel("Expected p-value (-log10 scale)")
    plt.ylabel("Observed p-value (-log10 scale)")
    plt.xlim(0, xMax)
    plt.ylim(0, yMax)
    plt.show()

Python makes it easy to make a [Q-Q (quantile-quantile) plot](https://en.wikipedia.org/wiki/Q-Q_plot).

In [ ]:
qqplot(gwas.query_variants('variants.map(v => va.linreg.pval[0]).collect()'), 
       5, 6)

## Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there's a confounder.

We didn't tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a [stratified](https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression. 

The [linreg](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.linreg) method can also take sample annotations to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don't have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The [pca](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.pca) method produces sample PCs in sample annotations, and can also produce variant loadings and global eigenvalues when asked.

In [ ]:
pca = common_vds.pca('sa.pca', k=5, eigenvalues='global.eigen')

In [ ]:
pprint(pca.globals)

In [ ]:
pprint(pca.sample_schema)

Now that we've got principal components per sample, we may as well plot them! Human history exerts a strong effect in genetic datasets. Even with a 50MB sequencing dataset, we can recover the major human populations.

In [ ]:
pca_table = pca.samples_table().to_pandas()
colors = {'AFR': 'green', 'AMR': 'red', 'EAS': 'black', 'EUR': 'blue', 'SAS': 'cyan'}
plt.scatter(pca_table["sa.pca.PC1"], pca_table["sa.pca.PC2"], 
            c = pca_table["sa.SuperPopulation"].map(colors), 
            alpha = .5)
plt.xlim(-0.6, 0.6)
plt.xlabel("PC1")
plt.ylabel("PC2")
legend_entries = [mpatches.Patch(color=c, label=pheno) for pheno, c in colors.items()]
plt.legend(handles=legend_entries, loc=2)
plt.show()

Now we can rerun our linear regression, controlling for the first few principal components and sample sex.

In [ ]:
pvals = (common_vds
        .annotate_samples_table(pca.samples_table(), expr='sa.pca = table.pca')
        .linreg(['sa.CaffeineConsumption'], covariates=['sa.pca.PC1', 'sa.pca.PC2', 'sa.pca.PC3', 'sa.isFemale'])
        .query_variants('variants.map(v => va.linreg.pval[0]).collect()'))

In [ ]:
qqplot(pvals, 5, 6)

In [ ]:
pvals = (common_vds
        .annotate_samples_table(pca.samples_table(), expr='sa.pca = table.pca')
        .linreg(['sa.CaffeineConsumption'], 
                covariates=['sa.pca.PC1', 'sa.pca.PC2', 'sa.pca.PC3', 'sa.isFemale'],
                use_dosages=True)
        .query_variants('variants.map(v => va.linreg.pval[0]).collect()'))

In [ ]:
qqplot(pvals, 5, 6)

That's more like it! We may not be publishing ten new coffee-drinking loci in _Nature_, but we shouldn't expect to find anything but the strongest signals from a dataset of 1000 individuals anyway. 

## Rare variant analysis

Hail doesn't yet have rare variant kernel-based methods, but we have [linear](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.linreg_burden) and [logistic](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.logreg_burden) burden tests. 

We won't be showing those here, though. Instead, we'll demonstrate how one can use the expression language to group and count by any arbitrary properties in variant or sample annotations.

In [ ]:
kt = (vds.genotypes_table()
         .aggregate_by_key(key_expr=['pop = sa.SuperPopulation', 'chromosome = v.contig'],
                           agg_expr=['n_het = g.filter(g => g.isHet()).count()']))

In [ ]:
kt.to_dataframe().show()

What if we want to group by minor allele frequency bin and hair color, and calculate the mean GQ?

In [ ]:
kt2 = (vds.genotypes_table()
          .aggregate_by_key(key_expr=['''maf_bin = if (va.qc.AF < 0.01) "< 1%" 
                                                   else if (va.qc.AF < 0.05) "1%-5%"
                                                   else "> 5%" ''',
                                     'purple_hair = sa.PurpleHair'],
                           agg_expr=['mean_gq = g.map(g => g.gq).stats().mean',
                                     'mean_dp = g.map(g => g.dp).stats().mean']))

In [ ]:
kt2.to_dataframe().show()

We've shown that it's easy to aggregate by a couple of arbitrary statistics. This specific examples may not provide especially useful pieces of information, but this same pattern can be used to detect effects of rare variation:

 - Count the number of heterozygous genotypes per gene by functional category (synonymous, missense, or loss-of-function) to estimate per-gene functional constraint
 - Count the number of singleton loss-of-function mutations per gene in cases and controls to detect genes involved in disease

## Eplilogue

Congrats! If you've made it this far, you're perfectly primed to read the [Overview](https://hail.is/hail/overview.html), look through the [Hail objects](https://hail.is/hail/types.html) representing many core concepts in genetics, and check out the many Hail functions defined in the [Python API](https://hail.is/hail/api.html). If you use Hail for your own science, we'd love to hear from you on [Gitter chat](https://gitter.im/hail-is/hail) or the [discussion forum](http://discuss.hail.is).

There's also a lot of functionality inside Hail that we didn't get to in this broad overview. Things like:

 - Flexible import and export to a variety of data and annotation formats (VCF, BGEN, PLINK, JSON, TSV, ...)
 - Simulation
 - Burden tests
 - Kinship and pruning (IBD, GRM, RRM)
 - Family-based tests and utilities
 - Distributed file system utilities
 - Interoperability with Python and Spark machine learning libraries
 - More!

For reference, here's the full workflow to all tutorial endpoints combined into one cell. It may take a minute! It's doing a lot of work.

In [ ]:
table = hc.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')
common_vds = (hc.read('data/1kg.vds')
              .annotate_samples_table(table, root='sa')
              .sample_qc()
              .filter_samples_expr('sa.qc.dpMean >= 4 && sa.qc.callRate >= 0.97')
              .filter_genotypes('''let ab = g.ad[1] / g.ad.sum() in
                         ((g.isHomRef && ab <= 0.1) ||
                          (g.isHet && ab >= 0.25 && ab <= 0.75) ||
                          (g.isHomVar && ab >= 0.9))''')
              .variant_qc()
              .filter_variants_expr('va.qc.AF > 0.01')
              .ld_prune(memory_per_core=512, num_cores=4))

pca = common_vds.pca('sa.pca', k=5, eigenvalues='global.eigen')
pvals = (common_vds
        .annotate_samples_table(pca.samples_table(), expr='sa.pca = table.pca')
        .linreg(['sa.CaffeineConsumption'], covariates=['sa.pca.PC1', 'sa.pca.PC2', 'sa.pca.PC3', 'sa.isFemale'])
        .query_variants('variants.map(v => va.linreg.pval[0]).collect()'))